## Iterators

Iteration is fundamental to data processing: programs apply computations to data series, from pixels to nucleotides. *If the data doesn’t fit in memory, we need to fetch the items lazily—one at a time and on demand*. That’s what an iterator does.  

Every standard collection in Python is iterable. *An iterable is an object that provides an iterator*, which Python uses to support operations like:
- `for` loops  
- List, dict, and set comprehensions  
- Unpacking assignments  
- Construction of collection instances  

## A Sequence of Words

In [5]:
# tag::SENTENCE_SEQ[]
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)  # `.findall` returns a list with all nonoverlapping 
                                            # matches of the regular expression, as a list of strings.

    def __getitem__(self, index):
        return self.words[index]  # return the word at the given index.

    def __len__(self):
        return len(self.words)

    def __repr__(self):
        """
        reprlib.repr is a utility function to generate abbreviated string
        representations of data structures that can be very large
        """
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
s = Sentence('"The time has come," the Walrus said,')
s

Sentence('"The time ha... Walrus said,')

In [6]:
for word in s:
    print(word)

The
time
has
come
the
Walrus
said


In [7]:
list(s)

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

In [9]:
s[0]

'The'

## Why Sequences Are Iterable: The `iter` Function

<span style="color:skyblue">***Whenever Python needs to iterate over an object `x`, it automatically calls `iter(x)`. The `iter` built-in function:***</span>
1. <span style="color:skyblue">*Checks whether the object implements `__iter__`, and calls that to obtain an iterator.*</span>
2. <span style="color:skyblue">*If `__iter__` is not implemented, but `__getitem__` is, then `iter()` creates an iterator that tries to fetch items by index, starting from 0 (zero). This is why all Python sequences are iterable: by definition, they all implement `__getitem__` (this is an extreme example of duct typing)*</span>
3. <span style="color:skyblue">*If that fails, Python raises `TypeError`, usually saying `'C'` object is not iterable, where `C` is the class of the target object.*</span>

In [13]:
class Spam:
    def __getitem__(self, i):
        print('->', i)
        raise IndexError()
    

spam_can = Spam()
iter(spam_can)

In [14]:
list(spam_can)

-> 0


[]

Although `spam_can` is iterable (its `__getitem__` could provide items), it is not recognized as such by an isinstance against `abc.Iterable`.

In [15]:
from collections import abc

isinstance(spam_can, abc.Iterable)

False

An object is considered iterable if it implements the `__iter__` method

In [16]:
class GooseSpam:
    def __iter__(self):
        pass

from collections import abc
print(f"{issubclass(GooseSpam, abc.Iterable) = }")

goose_spam_can = GooseSpam()
print(f"{isinstance(goose_spam_can, abc.Iterable) = }")

issubclass(GooseSpam, abc.Iterable) = True
isinstance(goose_spam_can, abc.Iterable) = True


**Note**: As of Python 3.10, the most accurate way to check whether an object x is iterable is to call `iter(x)` and handle a `TypeError` exception if it isn’t. This is more accurate than using `isinstance(x, abc.Iterable)`, because `iter(x)` also considers the legacy `__getitem__` method, while the Iterable ABC does not.

### Using `iter` with a Callable

We can call `iter()` with two arguments to create an iterator from a function or any callable object. In this usage, the first argument must be a callable to be invoked repeatedly (with no arguments) to produce values, and the second argument is a sentinel: a marker value which, when returned by the callable, causes the iterator to raise `StopIteration` instead of yielding the sentinel.

In [42]:
from random import randint

def d6():
    return randint(1, 6)

d6_iter = iter(d6, 1)

for roll in d6_iter:
    print(roll)

6
5
3


One useful application of the second form of `iter()` is to build a block-reader: 

```python
from functools import partial

with open('mydata.db', 'rb') as f:
    read64 = partial(f.read, 64)
    for block in iter(read64, b''):
        process_block(block)
```

## Iterables Versus Iterators

- **iterable**: Any object from which the `iter` built-in function can obtain an **iterator**. Objects implementing an `__iter__` method returning an iterator are iterable. Sequences are always iterable, as are objects implementing a `__getitem__` method that accepts 0-based indexes.

Python’s standard interface for an iterator has two methods:
- `__next__`: Returns the next item in the series, raising `StopIteration` if there are no more.
- `__iter__`: Returns `self`; this allows iterators to be used where an iterable is expected, for example, in a for loop.

<img src="../images/iterable-iterator.png" style="width: 50%;">.  


In [44]:
from collections.abc import Iterable
from abc import abstractmethod


def _check_methods(C, *methods):
    """
    traverses the `__mro__` of the class to check whether the methods
    are implemented in its base classes.
    """
    mro = C.__mro__
    for method in methods:
        for B in mro:
            if method in B.__dict__:
                if B.__dict__[method] is None:
                    return NotImplemented
                break
        else:
            return NotImplemented
    return True


class Iterator(Iterable):
    __slots__ = ()
    
    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise `StopIteration`'
        raise StopIteration
    
    def __iter__(self):
        return self
    
    @classmethod
    def __subclasshook__(cls, C):  # `__subclasshook__` supports structural type 
                                # checks with isinstance and issub class.
        if cls is Iterator:
            return _check_methods(C, '__iter__', '__next__')
        return NotImplemented
    

False

## Sentence Classes with `__iter__`

### Sentence Take #2: A Classic Iterator

The next `Sentence` implementation follows the blueprint of the classic `Iterator` design pattern from the Design Patterns book.

In [49]:
"""
Sentence: iterate over words using the Iterator Pattern, take #1

WARNING: the Iterator Pattern is much simpler in idiomatic Python;
see: sentence_gen*.py.
"""

# tag::SENTENCE_ITER[]
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        """
        The `__iter__` method is the only addition to the previous Sentence implementation. 
        This version has no `__getitem__`, to make it clear that the class is iterable 
        because it implements `__iter__`
        """
        return SentenceIterator(self.words)  # `__iter__` fulfills the iterable protocol by 
                                            # instantiating and returning an iterator.


class SentenceIterator:

    def __init__(self, words):
        self.words = words  # holds a reference to the list of words
        self.index = 0  # determines the next word to fetch

    def __next__(self):
        try:
            word = self.words[self.index]  # Get the word at self.index
        except IndexError:
            raise StopIteration()  # If there is no word at self.index, raise StopIteration
        self.index += 1  # Increment self.index
        return word  # Return the word

    def __iter__(self):  # Implement self.__iter__
        return self
# end::SENTENCE_ITER[]

def main():
    import sys
    import warnings

    word_number = 1
    with open('test.txt', 'rt', encoding='utf-8') as text_file:
        s = Sentence(text_file.read())
    for n, word in enumerate(s, 1):
        if n == word_number:
            print(word)
            break
    else:
        warnings.warn(f'last word is #{n}, {word!r}')

main()

Hello


### Don’t Make the Iterable an Iterator for Itself

<span style="color:orange">***A common cause of errors in building iterables and iterators is to confuse the two. To be clear: iterables have an `__iter__` method that instantiates a new iterator every time. Iterators implement a `__next__` method that returns individual items, and an `__iter__` method that returns `self`.***</span>

The “Applicability” section about the Iterator design pattern in the "Design Patterns" book says: Use the Iterator pattern
- to access an aggregate object’s contents without exposing its internal representation.
- to support multiple traversals of aggregate objects.
- to provide a uniform interface for traversing different aggregate structures (that is, to support polymorphic iteration).

To “support multiple traversals,” it must be possible to obtain multiple independent iterators from the same iterable instance, and each iterator must keep its own internal state, so a proper implementation of the pattern requires each call to iter(`my_iterable`) to create a new, independent, iterator. That is why we need the `SentenceIterator` class in this example.

### Sentence Take #3: A Generator Function

A Pythonic implementation of the same functionality uses a generator, avoiding all the work to implement the `SentenceIterator` class.

In [ ]:
"""
Sentence: iterate over words using a generator function
"""

# tag::SENTENCE_GEN[]
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

    def __iter__(self):
        for word in self.words:  # Iterate over self.words.
            # Yield the current word. Explicit return is not necessary; the function can
            # just “fall through” and return automatically. Either way, a generator function 
            # doesn’t raise `StopIteration`: it simply exits when it’s done producing values
            yield word  

# done! No need for a separate iterator class!

# end::SENTENCE_GEN[]

## How a Generator Works
Any Python function that has the `yield` keyword in its body is a generator function: a function which, when called, returns a generator object. In other words, a generator function is a generator factory.

In [50]:
def gen_123():
    yield 1
    yield 2
    yield 3

`gen_123` is a function object

In [51]:
gen_123

<function __main__.gen_123()>

But when invoked, `gen_123()` returns a generator object

In [52]:
gen_123()

<generator object gen_123 at 0x7f2c7fe13320>

Generator objects implement the `Iterator` interface, so they are also iterable

In [53]:
for i in gen_123():
    print(i)

1
2
3


In [54]:
g = gen_123()

Because `g` is an iterator, calling `next(g)` fetches the next item produced by `yield`. When the generator function returns, the generator object raises `StopIteration`.

In [55]:
next(g)

1

In [56]:
next(g)

2

In [57]:
next(g)

3

In [58]:
next(g)

StopIteration: 

Learn more about `yield`

In [60]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

gab = gen_AB()
next(gab)

start


'A'

In [61]:
next(gab)

continue


'B'

In [62]:
next(gab)

end.


StopIteration: 

In [63]:
for c in gen_AB():
    print('-->', c)

start
--> A
continue
--> B
end.


<span style="color:green">*That second version of `Sentence` is more concise than the first, but it’s not as lazy as it could be. Nowadays, laziness is considered a good trait, at least in programming languages and APIs. A lazy implementation postpones producing values to the last possible moment. This saves memory and may avoid wasting CPU cycles, too.*</span>

## Lazy Sentences

### Sentence Take #4: Lazy Generator
The `Iterator` interface is designed to be lazy: `next(my_iterator)` yields one item at a time. The opposite of lazy is eager: lazy evaluation and eager evaluation are technical terms in programming language theory.

Our `Sentence` implementations so far have not been lazy because the `__init__` eagerly builds a list of all words in the text, binding it to the `self.words` attribute. This requires processing the entire text, and the list may use as much memory as the
text itself.

In [ ]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text  # No need to have a words list

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        """
        The `re.finditer` function is a lazy version of `re.findall`. 
        Instead of a list, `re.finditer` returns a generator yielding 
        `re.MatchObject` instances on demand.
        """
        for match in RE_WORD.finditer(self.text):
            yield match.group()  # match.group() extracts the matched text from 
                                # the MatchObject instance.


### Sentence Take #5: Lazy Generator Expression

We can replace simple generator functions like the one in the previous `Sentence` class with a generator expression. As a list comprehension builds lists, a generator expression builds generator objects

In [13]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')


# the list comprehension eagerly iterates over the items 
# yielded by the generator object returned by gen_AB()
res1 = [x*3 for x in gen_AB()]
res1

start
continue
end.


['AAA', 'BBB']

In [14]:
for i in res1:
    print('-->', i)

--> AAA
--> BBB


In [15]:
# The generator expression returns res2, a generator object. The generator is not consumed here.
res2 = (x*3 for x in gen_AB())
res2

<generator object <genexpr> at 0x7a188999b850>

Only when the for loop iterates over `res2`, this generator gets items from `gen_AB`. Each iteration of the for loop implicitly calls `next(res2)`, which in turn calls `next()` on the generator object returned by `gen_AB()`, advancing it to the next yield.

In [16]:
for i in res2:
    print('-->', i)

start
--> AAA
continue
--> BBB
end.


Let's use a generator expression to further reduce the code in the `Sentence` class.

In [ ]:
import re
import reprlib
RE_WORD = re.compile(r'\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'

    def __iter__(self):
        """
        Instead of using a generator function, we use a generator
        expression to build a generator and then return it
        """
        return (match.group() for match in RE_WORD.finditer(self.text))

## When to Use Generator Expressions

A *generator expression* is a syntactic shortcut to create a generator without defining and calling a function. On the other hand, *generator functions* are more flexible: we can code complex logic with multiple statements, and we can even use them as coroutines.

<span style="color:green">***Rule**: If the generator expression spans more than a couple of lines, prefer to code a generator function for the sake of readability*</span>

## An Arithmetic Progression Generator

In [18]:
class ArithmeticProgression:
    def __init__(self, begin, step, end=None):
        self.begin = begin
        self.step = step
        self.end = end # None -> "infinite" series
    
    def __iter__(self):
        result_type = type(self.begin + self.step)  # Get the type of adding `self.begin` and `self.step`
                                                # e.g. if one is int and the other is float, result_type will be float.
        result = result_type(self.begin)  # makes a result with the same numeric value of self.begin, but
                                        # coerced to the type of the subsequent additions
        forever = self.end is None
        index = 0
        while forever or result < self.end:
            yield result  # The current result is produced.
            index += 1
            result = self.begin + self.step * index  # The next potential `result` is calculated. 
                                                # It may never be yielded, because the while loop may terminate

In [21]:
ap = ArithmeticProgression(0, 1, 3)
print(list(ap))
ap = ArithmeticProgression(1, .5, 3)
print(list(ap))

[0, 1, 2]
[1.0, 1.5, 2.0, 2.5]


Below is a generator function called `aritprog_gen` that does the same job as `ArithmeticProgression` but with less code

In [22]:
def aritprog_gen(begin, step, end=None):
    result = type(begin + step)(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index


ag = aritprog_gen(0, 1, 3)
print(list(ag))

[0, 1, 2]


`aritprog_gen` is elegant, but always remember: there are plenty of ready-to-use generators in the standard library, and the next section will show a shorter implementation using the itertools module.

### Arithmetic Progression with itertools

The itertools module in Python 3.10 has 20 generator functions that can be combined in a variety of interesting ways. For example, the `itertools.count` function returns a generator that yields numbers.

In [23]:
import itertools
gen = itertools.count(1, .5)
gen

count(1, 0.5)

In [30]:
next(gen)

4.0

`itertools.takewhile` function returns a generator that consumes another generator and stops when a given predicate evaluates to `False`

In [31]:
gen = itertools.takewhile(lambda n: n < 4, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5, 3.0, 3.5]

Leveraging `takewhile` and `count`, we can make `aritprog_gen` even more concise.

In [33]:
def aritprog_gen(begin, step, end=None):
    first = type(begin + step)(begin)
    ap_gen = itertools.count(first, step)
    if end is None:
        return ap_gen
    return itertools.takewhile(lambda n: n < end, ap_gen)

list(aritprog_gen(1, .5, 4))

[1.0, 1.5, 2.0, 2.5, 3.0, 3.5]

The point of the above example is: <span style="color:orange">*when implementing generators, know what is available in the standard library, otherwise there’s a good chance you’ll reinvent the wheel*</span>. That’s why the next section covers several ready-to-use generator functions.

## Generator Functions in the Standard Library

### Filtering generator functions
The first group contains the filtering generator functions: they yield a subset of items produced by the input iterable, without changing the items themselves.

- `itertools.compress(it, selector_it)`: Consumes two iterables in parallel; yields items from `it` whenever the corresponding item in `selector_it` is truthy
- `itertools dropwhile(predicate, it)`: Consumes `it`, skipping items while `predicate` computes truthy, then yields every remaining item (no further checks are made)
- (built-in) `filter(predicate, it)`: Applies `predicate` to each item of iterable, yielding the item if predicate(item) is truthy; if `predicate` is `None`, only truthy items are yielded
- `itertools.filterfalse(predicate, it)`: Same as filter, with the predicate logic negated: yields items whenever predicate computes falsy 
- `itertools.islice(it, stop)` or `islice(it, start, stop, step=1)`: Yields items from a slice of `it`, similar to `s[:stop]` or
`s[start:stop:step]` except it can be any iterable, and the operation is lazy
- `itertools.takewhile(predicate, it)`: Yields items while predicate computes truthy, then stops and no further checks are made

In [35]:
def vowel(c):
    return c.lower() in 'aeiou'

list(filter(vowel, 'Aardvark'))

['A', 'a', 'a']

In [36]:
list(itertools.filterfalse(vowel, 'Aardvark'))

['r', 'd', 'v', 'r', 'k']

In [37]:
list(itertools.dropwhile(vowel, 'Aardvark'))

['r', 'd', 'v', 'a', 'r', 'k']

In [38]:
list(itertools.takewhile(vowel, 'Aardvark'))

['A', 'a']

In [39]:
list(itertools.compress('Aardvark', (1, 0, 1, 1, 0, 1)))

['A', 'r', 'd', 'a']

In [40]:
list(itertools.islice('Aardvark', 4))

['A', 'a', 'r', 'd']

### Map generator functions

In [41]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
import itertools
list(itertools.accumulate(sample))

[5, 9, 11, 19, 26, 32, 35, 35, 44, 45]

In [42]:
list(itertools.accumulate(sample, min))

[5, 4, 2, 2, 2, 2, 2, 0, 0, 0]

In [43]:
list(itertools.accumulate(sample, max))

[5, 5, 5, 8, 8, 8, 8, 8, 9, 9]

In [44]:
import operator
list(itertools.accumulate(sample, operator.mul))

[5, 20, 40, 320, 2240, 13440, 40320, 0, 0, 0]

In [45]:
list(map(operator.mul, range(11), range(11)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [46]:
list(map(lambda a, b: (a, b), range(11), [2, 4, 8]))

[(0, 2), (1, 4), (2, 8)]

In [47]:
list(itertools.starmap(operator.mul, enumerate('albatroz', 1)))

['a', 'll', 'bbb', 'aaaa', 'ttttt', 'rrrrrr', 'ooooooo', 'zzzzzzzz']

### Merging generator function

### Expanding generator function

### Rearranging generator functions

## Iterable Reducing Functions

We have functions that take an iterable and return a single result:

- `all(it)`
- `any(it)`
- `max(it, [key=,] [default=])`
- `functools.reduce`
- `sum`

## Subgenerators with `yield from`

In [4]:
def subgen():
    yield 1.1
    yield 1.2

def gen():
    yield 1
    for i in subgen():
        yield i
    yield 2

for x in gen():
    print(x)

1
1.1
1.2
2


We can do the same thing as above with `yield from`

In [3]:
def subgen():
    yield 1.1
    yield 1.2

def gen():
    yield 1
    yield from subgen()
    yield 2

for x in gen():
    print(x)

1
1.1
1.2
2


Note that `yield from` pauses `gen`, and `subgen` takes over until it is exhausted.

We can also return values in subgenerators:

In [5]:
def sub_gen():
    yield 1.1
    yield 1.2
    return 'Done!'

def gen():
    yield 1
    result = yield from sub_gen()
    print('<--', result)
    yield 2

for x in gen():
    print(x)

1
1.1
1.2
<-- Done!
2


### Another example: `chain` multiple iterables

In [8]:
def chain(*iterables):
    for i in iterables:
        yield from i

s = 'ABC'
r = [0, 1, 2]
list(chain(s, r))

['A', 'B', 'C', 0, 1, 2]

### Bigger example: Traversing a Tree

**Step 1**: Only yield the class name

In [9]:
def tree(cls):
    yield cls.__name__

def display(cls):
    for cls_name in tree(cls):
        print(cls_name)
    
display(BaseException)

BaseException


**Step 2**: Yield also direct subclasses

In [10]:
def tree(cls):
    yield cls.__name__, 0  # yield also an int to indicate the level
    for sub_cls in cls.__subclasses__():
        yield sub_cls.__name__, 1


def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')

display(BaseException)

BaseException
    BaseExceptionGroup
    Exception
    GeneratorExit
    KeyboardInterrupt
    SystemExit
    CancelledError


**Step 3**: Delegate the subclasses to `sub_tree`

In [11]:
def tree(cls):
    yield cls.__name__, 0
    yield from sub_tree(cls)

def sub_tree(cls):
    for sub_cls in cls.__subclasses__():
        yield sub_cls.__name__, 1

def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')

display(BaseException)

BaseException
    BaseExceptionGroup
    Exception
    GeneratorExit
    KeyboardInterrupt
    SystemExit
    CancelledError


**Step 4**: `sub_tree` traverses level 1 and 2 (depth-first)

In [12]:
def tree(cls):
    yield cls.__name__, 0
    yield from sub_tree(cls)

def sub_tree(cls):
    for sub_cls in cls.__subclasses__():
        yield sub_cls.__name__, 1
        for sub_sub_cls in sub_cls.__subclasses__():
            yield sub_sub_cls.__name__, 2

def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')

display(BaseException)

BaseException
    BaseExceptionGroup
        ExceptionGroup
    Exception
        ArithmeticError
        AssertionError
        AttributeError
        BufferError
        EOFError
        ImportError
        LookupError
        MemoryError
        NameError
        OSError
        ReferenceError
        RuntimeError
        StopAsyncIteration
        StopIteration
        SyntaxError
        SystemError
        TypeError
        ValueError
        Warning
        ExceptionGroup
        _OptionError
        _Error
        error
        Error
        SubprocessError
        ArgumentError
        error
        ZMQBaseError
        Error
        PickleError
        _Stop
        TokenError
        StopTokenizing
        Error
        _GiveupOnSendfile
        Incomplete
        ClassFoundException
        EndOfBlock
        InvalidStateError
        LimitOverrunError
        QueueEmpty
        QueueFull
        TraitError
        Empty
        Full
        error
        error
        Retu

**Step 5**: There is a clear pattern here - We do a for loop to get the subclasses of level N. Each time around the loop, we yield a subclass of level N, then start another for loop to visit level N+1. Hence, we can do this recursively and replace the nested `for` loop with `yield from` on the same generator

In [14]:
def tree(cls):
    """
    Can traverses any depth
    """
    yield cls.__name__, 0
    yield from sub_tree(cls, 1)

def sub_tree(cls, level):
    """
    Recursive `sub_tree` has no if, but there is an implicit conditional in the for
    loop: if cls.__subclasses__() returns an empty list, the body of the loop is not
    executed, therefore no recursive call happens.
    """
    for sub_cls in cls.__subclasses__():
        yield sub_cls.__name__, level
        yield from sub_tree(sub_cls, level + 1)

def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')

display(BaseException)

BaseException
    BaseExceptionGroup
        ExceptionGroup
    Exception
        ArithmeticError
            FloatingPointError
            OverflowError
            ZeroDivisionError
                DivisionByZero
                DivisionUndefined
            DecimalException
                Clamped
                Rounded
                    Underflow
                    Overflow
                Inexact
                    Underflow
                    Overflow
                Subnormal
                    Underflow
                DivisionByZero
                FloatOperation
                InvalidOperation
                    ConversionSyntax
                    DivisionImpossible
                    DivisionUndefined
                    InvalidContext
        AssertionError
        AttributeError
            FrozenInstanceError
        BufferError
        EOFError
            IncompleteReadError
        ImportError
            ModuleNotFoundError
                PackageNotFoundE

**Step 6**: merge `tree` and `sub_tree` into a single generator

In [16]:
def tree(cls, level = 0):
    yield cls.__name__, level
    for sub_cls in cls.__subclasses__():
        yield from tree(sub_cls, level + 1)

def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')

display(BaseException)

BaseException
    BaseExceptionGroup
        ExceptionGroup
    Exception
        ArithmeticError
            FloatingPointError
            OverflowError
            ZeroDivisionError
                DivisionByZero
                DivisionUndefined
            DecimalException
                Clamped
                Rounded
                    Underflow
                    Overflow
                Inexact
                    Underflow
                    Overflow
                Subnormal
                    Underflow
                DivisionByZero
                FloatOperation
                InvalidOperation
                    ConversionSyntax
                    DivisionImpossible
                    DivisionUndefined
                    InvalidContext
        AssertionError
        AttributeError
            FrozenInstanceError
        BufferError
        EOFError
            IncompleteReadError
        ImportError
            ModuleNotFoundError
                PackageNotFoundE

## Generic Iterable Types

We can annotation functions that accept iterable arguments with `collections.abc.Iterable` (or `typing.Iterable` if you must
support Python 3.8 or earlier)


In [18]:
from collections.abc import Iterable

FromTo = tuple[str, str]

def zip_replace(text: str, changes: Iterable[FromTo]) -> str:
    for from_, to in changes:
        text = text.replace(from_, to)
    return text


The type `Iterator` can be used for generators coded as functions with `yield`, as well as iterators written “by hand” as classes with `__next__`. There is also a `collections.abc.Generator` type (and the corresponding deprecated typing.Generator) that we can use to annotate generator objects, but it is needlessly verbose for generators used as iterators.

In [17]:
from collections.abc import Iterator

def fibonacci() -> Iterator[int]:
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [19]:
from collections.abc import Iterator
from keyword import kwlist
from typing import TYPE_CHECKING

short_kw = (k for k in kwlist if len(k) < 5)
if TYPE_CHECKING:
    reveal_type(short_kw)

long_kw: Iterator[str] = (k for k in kwlist if len(k) >= 4)

if TYPE_CHECKING:
    reveal_type(long_kw)

## Classic Coroutines

Note that "classic coroutines" are generators used in a different way (different than the newer "native coroutines"). 

Generators are commonly used as iterators, but they can also be used as coroutines. A coroutine is really a generator function, created with the `yield` keyword in its body. And a coroutine object is physically a generator object.

The typing documentation describes the formal type parameters of `Generator` like
this: 
```python
Generator[YieldType, SendType, ReturnType]
```
where `SendType` is only relevant when the generator is used as a coroutine, and `ReturnType` is only meaningful to annotate a coroutine, because iterators don’t return values like regular functions. The `YieldType` is the type of the value returned by a call to `next(it)`.
The `Generator` type has the same type parameters as `typing.Coroutine`:
```python
Coroutine[YieldType, SendType, ReturnType]
```

Things to remember about generators and coroutines:
- <span style="color:skyblue">*Generators produce data for iteration*</span>
- <span style="color:skyblue">*Coroutines are consumers of data*</span>
- <span style="color:skyblue">*To keep your brain from exploding, don’t mix the two concepts together*</span>
- <span style="color:skyblue">*Coroutines are not related to iteration*</span>
- <span style="color:skyblue">*There is a use of having `yield` produce a value in a coroutine, but it’s not tied to iteration.*</span>

## Summary


Iteration is so deeply embedded Python, and the integration of the `Iterator` pattern in the semantics of Python is a prime example of how design patterns are not equally applicable in all programming languages. 

In this chapter, we built a few versions of a class to iterate over individual words in text files that may be very long. We saw how Python uses the `iter()` built-in to create iterators from sequence-like objects. We build a classic iterator as a class with `__next__()`, and then we used generators to make each successive refactoring of the `Sentence` class more concise and readable.

We then coded a generator of arithmetic progressions and showed how to leverage the `itertools` module to make it simpler. An overview of most general-purpose generator functions in the standard library followed.

We then studied `yield from` expressions in the context of simple generators with the chain and tree examples.

The last major section was about classic coroutines, a topic of waning importance after native coroutines were added in Python 3.5. Although difficult to use in practice, classic coroutines are the foundation of native coroutines, and the `yield from` expression is the direct precursor of `await`.
Also covered were type hints for `Iterable`, `Iterator`, and `Generator` types—with the latter providing a concrete and rare example of a contravariant type parameter.